# Predicting Financial Performance with Standardized Data and Hierarchical Forecasting

## We can improve net income forecasts by using industry and line item level detail as inputs to a hierarchical forecasting algorithm.

### Following https://github.com/Nixtla/hierarchicalforecast/blob/main/nbs/examples/TourismSmall.ipynb

In [ ]:
# %#pip install statsforecast hierarchicalforecast utilsforcast

In [1]:
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive, HoltWinters, SeasonalNaive
import pandas as pd

from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation, mse, rel_mse, mqloss
from hierarchicalforecast.methods import TopDown, BottomUp, MinTrace, MiddleOut
from hierarchicalforecast.utils import aggregate, HierarchicalPlot
from utilsforecast.preprocessing import fill_gaps
import numpy as np

import pickle
from datetime import date, datetime
from sklearn.metrics import mean_absolute_percentage_error
from qgridnext import show_grid
from importlib import reload
import re
import pickle
from importlib import reload

C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsforecast\core.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsforecast\utils.py:236: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "ds": pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M"),


In [21]:
TEST_PERIODS = 1

In [4]:
# Constructed by the build_input_data.ipynb script

Y_df_load = pd.read_parquet("test_train_data.parquet")

## Build Training and Testing DataSets

In [22]:
levels = [
    ["market"],
    ["market", "top_level_SIC_code"],
    ["market", "top_level_SIC_code", "second_level_SIC_code"],
    ["market", "top_level_SIC_code", "second_level_SIC_code", "ticker"],
    ["market", "top_level_SIC_code", "second_level_SIC_code", "ticker", 'metric']
]

In [23]:
Y_df, S_df, tags = aggregate(df=Y_df_load, spec=levels)

In [ ]:
#S_df.to_parquet('S_df.parquet')

In [ ]:
#with open('tags.pickle', 'wb') as out:
#    pickle.dump(tags, out)

In [24]:
Y_df = Y_df.reset_index()

In [25]:
Y_test_df = Y_df.groupby("unique_id").tail(TEST_PERIODS)
Y_train_df = Y_df.drop(Y_test_df.index)
Y_test_df = Y_test_df.set_index("unique_id")

## Base Forecasts

In [26]:
season_length = 4
fcst = StatsForecast(
    models=[AutoARIMA(season_length=season_length)],
    freq="QE",
    fallback_model=SeasonalNaive(season_length=season_length),
    n_jobs=1
)

In [27]:
Y_hat_df = fcst.forecast(
    df=Y_train_df,
    h=TEST_PERIODS,
    fitted=True,
)

C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsforecast\arima.py:1246: RuntimeWarning: divide by zero encountered in long_scalars
  fit["aicc"] = fit["aic"] + 2 * npar * (npar + 1) / (nstar - npar - 1)
C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsforecast\arima.py:1246: RuntimeWarning: divide by zero encountered in long_scalars
  fit["aicc"] = fit["aic"] + 2 * npar * (npar + 1) / (nstar - npar - 1)
C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsforecast\arima.py:1246: RuntimeWarning: divide by zero encountered in long_scalars
  fit["aicc"] = fit["aic"] + 2 * npar * (npar + 1) / (nstar - npar - 1)
C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsforecast\arima.py:1246: RuntimeWarning: divide by zero encountered in long_scalars
  fit["aicc"] = fit["aic"] + 2 * npar * (npar + 1) / (nstar - npar - 1)
C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-pack

C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsforecast\arima.py:894: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsforecast\arima.py:894: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsforecast\arima.py:894: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsforecast\arima.py:894: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsforecast\arima.py:1246: RuntimeWarning: divide by zero encountered in long_scalars
  fit["aicc"] = fit["aic"] + 2 * npar * (npar + 1) / (nstar - npar - 1)
C:\Users\andre\AppData\Local\Programs\Python\Python311\

KeyboardInterrupt: 

In [ ]:
Y_hat_df.to_parquet('Y_hat_df.parquet')

In [8]:
Y_hat_df = pd.read_parquet('Y_hat_df.parquet')

In [ ]:
Y_fitted_df = fcst.forecast_fitted_values()

In [ ]:
Y_fitted_df.to_parquet('Y_fitted_df.parquet')

In [19]:
Y_fitted_df = pd.read_parquet('Y_fitted_df.parquet')

In [20]:
Y_fitted_df

,ds,y,AutoARIMA
unique_id,,,
0,2010-03-31,6419.000000,6412.581055
0,2010-06-30,8484.000000,8475.515625
0,2010-09-30,8986.602539,8977.616211
0,2010-12-31,7414.144043,7406.729980
0,2011-03-31,7809.570801,6898.240234
...,...,...,...
8/9/NVRI,2023-03-31,468.302002,464.677063
8/9/NVRI,2023-06-30,495.653015,465.793488
8/9/NVRI,2023-09-30,520.168030,503.294006


In [15]:
fcst.plot(
    Y_train_df,
    Y_hat_df,
    models=["AutoARIMA"],
    unique_ids=["market"],
)

NameError: name 'fcst' is not defined

## Hierarchical reconciliation

In [2]:
hrec = HierarchicalReconciliation(
    reconcilers=[
        # BottomUp(),
     #   hierarchicalforecast.methods.MiddleOut(
     #       middle_level="market/top_level_SIC_code/second_level_SIC_code/ticker",
     #       top_down_method="forecast_proportions",
     #   ),
        MinTrace(method="ols"),
        # MinTrace(method="ols"),
        #    TopDown(method='forecast_proportions')
    ]
)

In [9]:
# Some records did not make it into Y_hat_df, I think because they are 0

missing_records = S_df.index.unique().difference(Y_hat_df.index.unique())
S_df_minus_missing_records = S_df[~S_df.index.isin(missing_records)]

In [16]:
Y_hat_df

,ds,AutoARIMA
unique_id,,
market,2024-06-30,9.331184e+06
market,2024-09-30,9.393458e+06
market/0,2024-06-30,1.647985e+04
market/0,2024-09-30,1.514611e+04
market/0/1,2024-06-30,1.305447e+04
...,...,...
market/8/9/NVRI/Other,2024-09-30,0.000000e+00
market/8/9/NVRI/Revenue,2024-06-30,5.362125e+02
market/8/9/NVRI/Revenue,2024-09-30,5.428422e+02


In [17]:
Y_fitted_df

,ds,y,AutoARIMA
unique_id,,,
0,2010-03-31,6419.000000,6412.581055
0,2010-06-30,8484.000000,8475.515625
0,2010-09-30,8986.602539,8977.616211
0,2010-12-31,7414.144043,7406.729980
0,2011-03-31,7809.570801,6898.240234
...,...,...,...
8/9/NVRI,2023-03-31,468.302002,464.677063
8/9/NVRI,2023-06-30,495.653015,465.793488
8/9/NVRI,2023-09-30,520.168030,503.294006


In [12]:
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S=S_df_minus_missing_records, tags=tags)

Exception: Check `Y_hat_df`, `Y_df` series difference, Y_hat\Y=30609, Y\Y_hat=1

In [ ]:
Y_rec_df

In [ ]:
Y_rec_df.to_parquet('Y_rec_df.parquet')

In [ ]:
plot_df = pd.concat(
    [
        Y_df.reset_index().set_index(["unique_id", "ds"]),
        Y_rec_df.set_index("ds", append=True),
    ],
    axis=1,
)
plot_df = plot_df.reset_index("ds")

In [ ]:
hplot = HierarchicalPlot(S=S_df, tags=tags)
hplot.plot_series(
    series="market",
    Y_df=plot_df,
    models=[
        "y",
        "AutoARIMA",
        "AutoARIMA/TopDown_method-forecast_proportions",
    ],
    level=[80],
)

In [ ]:
hplot = HierarchicalPlot(S=S_df, tags=tags)
difference_df = Y_hat_df.merge(Y_test_df, on=["unique_id", "ds"])[
    ["ds", "AutoARIMA", "y"]
]

# hplot.plot_hierarchical_predictions_gap(Y_hat_df.rename({"Naive": "y"}, axis=1))

hplot.plot_hierarchical_predictions_gap(difference_df)

In [ ]:
show_grid(Y_rec_df)

# Evaluation
## https://github.com/Nixtla/hierarchicalforecast/blob/2296c259542dbd906cfba4b8345c3b72148dad79/nbs/examples/TourismSmall.ipynb#L463

In [ ]:
evaluator = HierarchicalEvaluation(evaluators=[mse])
evaluation = evaluator.evaluate(
    Y_hat_df=Y_rec_df,
    Y_test_df=Y_test_df,
    tags=tags,
    benchmark="AutoARIMA",
)

In [ ]:
# Deal with the missing values again

tags["market/top_level_SIC_code/second_level_SIC_code/ticker/metric"] = tags[
    "market/top_level_SIC_code/second_level_SIC_code/ticker/metric"
][
    ~tags["market/top_level_SIC_code/second_level_SIC_code/ticker/metric"].isin(
        missing_records
    )
]

In [ ]:
evaluation.T

In [ ]:
show_grid(
    Y_rec_df.set_index(["ds"], append=True)
    .sub(Y_test_df.set_index(["ds"], append=True)["y"], axis="index")[
        ["AutoARIMA", "AutoARIMA/BottomUp"]
    ]
    .query('ds == "2024-06-30"')
)